In [108]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso,LassoCV,Ridge,RidgeCV
from scipy.stats import zscore
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
import pickle

In [109]:
df=pd.read_excel('C:/Users/Admin/OneDrive/Desktop/EDUCATION/DATA SET_DATATRAINED/RESURANT DATA SET/Data_Train.xlsx')
df.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300


In [110]:
print(df.shape,df.columns,df.dtypes,sep='/////')

(12690, 9)/////Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES', 'COST'],
      dtype='object')/////TITLE            object
RESTAURANT_ID     int64
CUISINES         object
TIME             object
CITY             object
LOCALITY         object
RATING           object
VOTES            object
COST              int64
dtype: object


In [111]:
df.isnull().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

In [112]:
for i in df:
    print(i,df[i].unique().size)

TITLE 113
RESTAURANT_ID 11892
CUISINES 4155
TIME 2689
CITY 360
LOCALITY 1417
RATING 33
VOTES 1848
COST 86


In [113]:
df_n=df.drop(columns=['RESTAURANT_ID'])

In [114]:
a=df_n.groupby('VOTES')
a.first()

,TITLE,CUISINES,TIME,CITY,LOCALITY,RATING,COST
VOTES,,,,,,,
10 votes,CAFÉ,Cafe,10am – 11pm (Mon-Sun),Kochi,Elamkulam,3.1,400
100 votes,CASUAL DINING,"North Indian, South Indian, Biryani, Chinese","12noon – 11:45pm (Mon, Tue, Wed, Thu, Sun), 12...",Secunderabad,Uppal,3.8,600
1000 votes,FINE DINING,"Continental, North Indian, South Indian, Italian",24 Hours (Mon-Sun),New Delhi,Khan Market,4.0,3500
1001 votes,"MICROBREWERY,CASUAL DINING","American, Italian, Chinese",12noon – 12midnight (Mon-Sun),Gurgaon,DLF Cyber City,4.1,1750
1002 votes,CASUAL DINING,"Chinese, Asian, Thai, Vietnamese, Momos",11:30am – 11:30pm (Mon-Sun),Powai,Hiranandani Gardens,3.9,800
...,...,...,...,...,...,...,...
993 votes,"BAKERY,QUICK BITES","Bakery, Desserts, Fast Food, Salad",10am – 9:30pm (Mon-Sun),New Delhi,Kailash Colony,4.2,300
996 votes,"CASUAL DINING,BAR","Goan, Mangalorean, Pizza","12noon – 11pm (Mon, Tue, Wed, Thu, Sun), 12noo...",Bangalore,Ulsoor,4.2,1000
997 votes,"CASUAL DINING,BAR","South Indian, North Indian, Chinese, Biryani, BBQ","12noon – 11:30pm (Mon, Tue, Wed, Thu, Sun), 12...",Jubilee Hills,near Andhra Prabha office,4.1,1200


In [115]:
df_n['VOTES']=df_n['VOTES'].str.strip('votes')

In [116]:
df_n['VOTES']=df_n['VOTES'].astype(float)

In [117]:
print(df_n['RATING'].unique())
df_n['RATING']=df_n['RATING'].dropna(inplace=False)

['3.6' '4.2' '3.8' '4.1' '4.0' '4.3' '3.9' '3.3' '3.4' '-' '4.5' '3.5'
 '4.4' '2.7' '3.7' '4.7' 'NEW' '3.1' '2.5' '4.6' '2.8' nan '3.0' '3.2'
 '2.6' '2.9' '4.9' '4.8' '2.4' '2.3' '2.0' '2.1' '2.2']


In [118]:
df_n.shape

(12690, 8)

In [119]:
df_n.isnull().sum()

TITLE          0
CUISINES       0
TIME           0
CITY         112
LOCALITY      98
RATING         2
VOTES       1204
COST           0
dtype: int64

In [120]:
df_n.dtypes

TITLE        object
CUISINES     object
TIME         object
CITY         object
LOCALITY     object
RATING       object
VOTES       float64
COST          int64
dtype: object